In [1]:
!pip install sodapy

distributed 1.21.8 requires msgpack, which is not installed.


In [2]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
# Dates and location formatted
results = client.get("me59-5fac", limit=50000)

# Convert to pandas DataFrame
complaints = pd.DataFrame.from_records(results)

In [3]:
# download remaining (limit 50000 / call)
start = 50000
while results:
    print(start)
    results = client.get("me59-5fac", limit=50000, offset=start)
    complaints = complaints.append(pd.DataFrame.from_records(results))
    start += 50000


50000
100000
150000


In [4]:
# Drop rows with missing data
complaints.dropna(subset=["latitude", "longitude", "creation_date"], inplace=True)

In [5]:
# Filter by status
complaints = complaints[complaints.status.isin(["Completed", "Open"])]

In [7]:
import os.path
root_path = os.path.dirname(os.getcwd())

# Save result
complaints.to_csv(os.path.join(root_path, "DATA/sanitation_complaints.csv"), index=False)